<a href="https://colab.research.google.com/github/Kent-mak/Anti-Hate-dashboard/blob/filter_model/Civil%20Regression%20BERT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install torch datasets transformers peft

  Using cached datasets-2.19.1-py3-none-any.whl (542 kB)
  Using cached peft-0.11.0-py3-none-any.whl (251 kB)
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.19.3-py3-none-manylinux1_x86_64.whl (166.0 MB)
  Using cached nvidia_nvtx

Define Model with LoRA for regression

In [3]:
import torch
if torch.cuda.is_available():
    device_name = torch.device("cuda")
else:
    device_name = torch.device('cpu')
print("Using {}.".format(device_name))

Using cuda.


In [4]:

import torch.nn as nn


class RegressionModel(nn.Module):
    def __init__(self, base_model, num_labels=7):
        super(RegressionModel, self).__init__()
        self.model = base_model
        self.dropout = nn.Dropout(0.1)
        self.regressor = nn.Linear(self.model.config.hidden_size, num_labels)

    def forward(self, input_ids, attention_mask=None, labels=None):
        outputs = self.model(input_ids, attention_mask=attention_mask)
        pooled_output = outputs[0][:, 0]
        pooled_output = self.dropout(pooled_output)
        logits = self.regressor(pooled_output)

        if labels is not None:
            loss_fct = nn.MSELoss()
            loss = loss_fct(logits, labels)
            return loss, logits
        else:
            return logits


In [5]:
from peft import LoraConfig, get_peft_model
from transformers import AutoModel

model_name = "distilbert-base-uncased"
base_model = AutoModel.from_pretrained(model_name)

lora_config = LoraConfig(
    r=4,                      # Rank of the low-rank matrices
    lora_alpha=32,            # Scaling factor
    lora_dropout=0.1,         # Dropout probability for LoRA layers
    target_modules=["attention.q_lin", "attention.k_lin", "attention.v_lin"]  # Modules to which LoRA will be applied
)

model_with_lora = get_peft_model(base_model, lora_config)


/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

In [6]:
# Load the tokenizer
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained(model_name)

# Integrate LoRA with the custom regression model
model = RegressionModel(model_with_lora)

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

Work with the dataset

In [7]:
from datasets import load_dataset

# Load your dataset
# dataset = load_dataset('google/civil_comments')
training_set = load_dataset('google/civil_comments', split= 'train[:2%]')
validation_set = load_dataset('google/civil_comments', split= 'validation[:2%]')
test_set = load_dataset('google/civil_comments', split= 'test[:2%]')
print(training_set)

Generating train split:   0%|          | 0/1804874 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/97320 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/97320 [00:00<?, ? examples/s]

Dataset({
    features: ['text', 'toxicity', 'severe_toxicity', 'obscene', 'threat', 'insult', 'identity_attack', 'sexual_explicit'],
    num_rows: 36097
})


In [8]:
# Define the preprocessing function
def preprocess_function(examples):
    inputs = tokenizer(examples['text'], truncation=True, padding='max_length')
    labels = torch.tensor(list(zip(
        examples['toxicity'],
        examples['severe_toxicity'],
        examples['obscene'],
        examples['threat'],
        examples['insult'],
        examples['identity_attack'],
        examples['sexual_explicit']
    )), dtype=torch.float32)

    inputs["labels"] = labels
    return inputs

# Apply the preprocessing function to each split
# encoded_dataset = dataset.map(preprocess_function, batched=True, remove_columns=dataset["train"].column_names)
encoded_training_set = training_set.map(preprocess_function, batched=True, remove_columns=training_set.column_names)
encoded_validation_set = validation_set.map(preprocess_function, batched=True, remove_columns=validation_set.column_names)
encoded_test_set = test_set.map(preprocess_function, batched=True, remove_columns=test_set.column_names)
print(encoded_training_set)



Map:   0%|          | 0/36097 [00:00<?, ? examples/s]

Map:   0%|          | 0/1946 [00:00<?, ? examples/s]

Map:   0%|          | 0/1946 [00:00<?, ? examples/s]

Dataset({
    features: ['input_ids', 'attention_mask', 'labels'],
    num_rows: 36097
})


Train the model

In [9]:
from transformers import Trainer, TrainingArguments

class CustomTrainer(Trainer):
    def compute_loss(self, model, inputs, return_outputs=False):
      # print(inputs)
      labels = inputs.get("labels").to(device_name)
      model.to(device_name)
      loss, logits = model(input_ids=inputs["input_ids"], attention_mask=inputs["attention_mask"], labels=labels)
      return (loss, logits) if return_outputs else loss


In [ ]:


# Training arguments
training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=32,
    per_device_eval_batch_size=32,
    num_train_epochs=3,
    weight_decay=0.01,
)

# Trainer
trainer = CustomTrainer(
    model=model,
    args=training_args,
    train_dataset=encoded_training_set,
    eval_dataset=encoded_validation_set,
)

# Fine-tune the model
trainer.train()
test_results = trainer.evaluate(encoded_test_set)
print(f"Test Results: {test_results}")

Epoch,Training Loss,Validation Loss


Epoch,Training Loss,Validation Loss


In [ ]:
baseRegressionModel = RegressionModel(base_model)

training_args = TrainingArguments(
    output_dir="./base_results",
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=32,
    per_device_eval_batch_size=32,
    num_train_epochs=3,
    weight_decay=0.01,
)

# Trainer
base_trainer = CustomTrainer(
    model=baseRegressionModel,
    args=training_args
)

base_test_results = base_trainer.evaluate(encoded_test_set)
print(f"Test Results: {base_test_results}")

Test Results: {'eval_loss': 0.04703187197446823, 'eval_runtime': 9.7189, 'eval_samples_per_second': 100.114, 'eval_steps_per_second': 3.19}


In [ ]:
torch.save(model.state_dict(), "./model.pt")

cp: -r not specified; omitting directory '/content/results'


In [ ]:
from transformers import AutoTokenizer

loaded_model = RegressionModel()
loaded_model.load_state_dict(torch.load("./model.pt"))
loaded_model.eval()

new_tokenizer = AutoTokenizer.from_pretrained("./model.pt")
example_text = 'haha you guys are a bunch of losers.'
input = new_tokenizer(example_text, return_tensors="pt")

with torch.no_grad():
  loss, logits = loaded_model(**input)

print(logits)
print(loss)